In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
# Carregando variaveis de ambiente

from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Define p array de perguntas
perguntas = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em 'Os Sertões'? Como esses aspectos refletem a visão do autor sobre o Brasil da época?"
]

In [ ]:
# Load dos modelos ( Embeddings e LLM)

embeddings_model = OpenAIEmbeddings()

llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=200) 

In [5]:
# Carrega o PDF

pdf_link = "../os-sertoes.pdf"

loader = PyPDFLoader(pdf_link,extract_images=False)

pages = loader.load_and_split()

In [ ]:
# Separar em chunkns(pedaçõs de documentos)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 4000,
    chunk_overlap = 20,
    length_function = len,
    add_start_index = True
)

chunks = text_splitter.split_documents(pages)

In [ ]:
# Salvar no Vector DB

db = Chroma.from_documents(chunks,embedding=embeddings_model, persist_directory="text_index")

In [ ]:
# Carregar DB

vectordb = Chroma(persist_directory="text_index", embedding_function=embeddings_model)

# Load Retriever

retriever =  vectordb.as_retriever(search_kwargs={"k":3})

# Construcao da cadeia de prompt para chamada do LLM



In [ ]:
# Prompt
prompt = PromptTemplate.from_template(
    """
    Você é um experiente analisador de obras literárias. Responda a pergunta abaixo utilizando o contexto informado.
    Query:
    {question}
    Context:
    {context}
    """
)

In [ ]:
# Cria a cadeia para consulta
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:

for p in perguntas:
    resposta = chain.invoke(p)
    print(f"Pergunta:\n{p}\n")
    print(f"Resposta:\n{resposta}")
    print("-" * 80)